## Markers plot

In [58]:
# load libraries
require(RIdeogram)

# From the Rstudio console set working directory "plot_markers/" folder:
# setwd("/path/to/support_protocol2/plot_markers/")


karyotype_file <- "./busco_protocol-main/support_protocol2/plot_markers/data/karyotype/Dmel_karyotype.txt"  # provide path to karyotype file 
coordinates <- "./busco_protocol-main/support_protocol2/plot_markers/data/full_table/Dmel_full_table.tsv" # provide path to markers' coordinates
out_name <- "Dmel" # sample name

# load karyotype
karyotype <- read.csv(karyotype_file, sep="\t", header = TRUE, stringsAsFactors = F)

# load mapping busco_coordinates.txt
busco_mappings <- read.csv(coordinates, sep="\t", header = FALSE, stringsAsFactors = F)

colnames(busco_mappings)[1] <- "Status"
colnames(busco_mappings)[2] <- "Chr"
colnames(busco_mappings)[3] <- "Start"
colnames(busco_mappings)[4] <- "End"

busco_mappings$Type <- "BUSCO_marker"
busco_mappings$Shape <- "circle" 

# change status in color
busco_mappings$Status <- gsub("Complete", '2dacd6', busco_mappings$Status)
busco_mappings$Status <- gsub("Duplicated", '0a0e1a', busco_mappings$Status)
busco_mappings$Status <- gsub("Fragmented", 'eded13', busco_mappings$Status)
colnames(busco_mappings)[1] <- "color"

busco_mappings <- busco_mappings[, c(2,3,4,1)]

ideogram(karyotype = karyotype, label = busco_mappings, label_type = "marker", output = "marker.svg")
# convert to png
convertSVG("marker.svg", device = "png")

ERROR: Error in read_data(svg): Argument 'svg' or 'css' must be a file path, url, or raw vector.


In [1]:
install.packages("RIdeogram")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [1]:
require(RIdeogram)
data(human_karyotype, package="RIdeogram")
data(gene_density, package="RIdeogram")
data(Random_RNAs_500, package="RIdeogram")

Loading required package: RIdeogram



In [3]:
data(liriodendron_karyotype, package="RIdeogram")

In [4]:
head(liriodendron_karyotype)

,Chr,Start,End
,<int>,<int>,<int>
1,1,0,118073833
2,2,0,98364873
3,3,0,207093695
4,4,0,50051714
5,5,0,45443526
6,6,0,35772468


In [27]:
human_karyotype <- human_karyotype[1:10,]

In [3]:
ideogram(karyotype = human_karyotype, label = Random_RNAs_500, label_type = "marker")
convertSVG("chromosome.svg", device = "png")

ERROR: Error in read_data(svg): Argument 'svg' or 'css' must be a file path, url, or raw vector.


## Syntenies plot

In [1]:
# load libraries
require(RIdeogram)

# From the Rstudio console set working directory "plot_syntenies/" folder
# setwd("/{path_to/support_protocol2/plot_syntenies/")

# paths are already set for the test data
species1 <- "Dmel" # assign a name to species1, e.g. "Dmela"
karyotype1_file <- "./busco_protocol-main/support_protocol2/plot_syntenies/data/karyotype/Dmel_karyotype.txt" # karyotype of species 1
coord1_file <- "./busco_protocol-main/support_protocol2/plot_syntenies/data/full_table/Dmel_coordinates.tsv"   # buscos coordinates

species2 <- "Dpseudo" # assign a name to species2, e.g. "Dpseudo"
karyotype2_file <- "./busco_protocol-main/support_protocol2/plot_syntenies/data/karyotype/Dpseudo_karyotype.txt" # karyotype of species 2
coord2_file <- "./busco_protocol-main/support_protocol2/plot_syntenies/data/full_table/Dpseudo_coordinates.tsv" # buscos coordinates

### karyotype 1 --------
karyotype1 <- read.csv(file=karyotype1_file, sep="\t", header = TRUE, stringsAsFactors = F)

### karyotype 2 --------
karyotype2 <- read.csv(file=karyotype2_file, sep="\t", header = TRUE, stringsAsFactors = F)

###merge  two kayotypes ----
karyotype_all <- rbind(karyotype1, karyotype2, make.row.names=FALSE)

karyotype_all$fill <- "2dacd6" # 2dacd6

karyotype_all[karyotype_all$species==species1, ]$fill <- "8AFFBB"
karyotype_all[karyotype_all$species==species2, ]$fill <- "95FF7D"

karyotype_all <- karyotype_all[, c(1,2,3,7,4,5,6)]
karyotype_all <- as.data.frame(karyotype_all, stringsAsFactors = F )

### add coordinates ----

coord1 <- read.csv(file=coord1_file, sep="\t", header = FALSE, stringsAsFactors = F)
coord2 <- read.csv(file=coord2_file, sep="\t", header = FALSE, stringsAsFactors = F)

coord1 <- coord1[coord1$V2=="Complete", ]
coord2 <- coord2[coord2$V2=="Complete", ]

# only keep results for selected sequences
coord1 <- coord1[coord1$V3 %in% karyotype1$Chr,]
coord2 <- coord2[coord2$V3 %in% karyotype2$Chr,]

sinteny_coord <- merge(coord1, coord2, by="V1")
head(sinteny_coord)

sinteny_coord <- sinteny_coord[, c(-1, -2, -6)]
colnames(sinteny_coord) <- c("Species_1", "Start_1", "End_1", "Species_2", "Start_2", "End_2")
sinteny_coord$fill <- "cccccc"

# convert names into integer according to position in karyotype_all
chromosomes_species1 <- karyotype1$Chr
chromosomes_species2 <- karyotype2$Chr

for(seq in chromosomes_species1) {
  index <- which(grepl(seq, karyotype_all$Chr))
  sinteny_coord$Species_1 <- gsub(seq, index, sinteny_coord$Species_1)
}

for(seq in chromosomes_species2) {
  index <- which(grepl(seq, karyotype_all$Chr))
  index_species_2 <- index - length(chromosomes_species1)
  sinteny_coord$Species_2 <- gsub(seq, index_species_2, sinteny_coord$Species_2)
}

# You can assign different colors to lines linking different chromosomes, 
# e.g. to color the lines as in the protocol figure, uncomment these lines:
# sinteny_coord$fill <- "2fbeed"
# sinteny_coord[sinteny_coord$Species_1==1,]$fill <- "FF6E63" #, "9c7a14"
# sinteny_coord[sinteny_coord$Species_1==4,]$fill <- "EEFF70" # 96323c"

sinteny_coord <- sinteny_coord[with(sinteny_coord, order(fill)),]

write.table(sinteny_coord, file = paste0(species1, "_", species2, "_sinteny.txt"), quote = FALSE, 
            row.names = FALSE, col.names = TRUE, sep = "\t")
write.table(karyotype_all, file = paste0(species1, "_", species2, "_karyotype_all.txt"), quote = FALSE, 
            row.names = FALSE, col.names = TRUE, sep = "\t")

sinteny_coord <- read.table(paste0(species1, "_", species2, "_sinteny.txt"), sep = "\t", header = T, stringsAsFactors = F)

# Plot with ideogram function
ideogram(karyotype = karyotype_all, synteny = sinteny_coord,  output = paste0(species1, "_", species2, "_sinteny.svg"))
convertSVG(paste0(species1, "_", species2, "_sinteny.svg"), device = "png")

Loading required package: RIdeogram



V1,V2.x,V3.x,V4.x,V2.y,V3.y,V4.y
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>


ERROR: Error in names(x) <- value: 'names' attribute [6] must be the same length as the vector [4]


In [5]:
require(RIdeogram)
data(human_karyotype, package="RIdeogram")

head(human_karyotype)

,Chr,Start,End,CE_start,CE_end
,<chr>,<int>,<int>,<int>,<int>
1,1,0,248956422,122026459,124932724
2,2,0,242193529,92188145,94090557
3,3,0,198295559,90772458,93655574
4,4,0,190214555,49712061,51743951
5,5,0,181538259,46485900,50059807
6,6,0,170805979,58553888,59829934


In [16]:
data(human_karyotype, package="RIdeogram")
data(gene_density, package="RIdeogram")
data(Random_RNAs_500, package="RIdeogram")

In [59]:
data(karyotype_dual_comparison, package="RIdeogram")
#table(karyotype_dual_comparison$species)
data(synteny_dual_comparison, package="RIdeogram")

In [60]:
ideogram(karyotype = karyotype_dual_comparison, synteny = synteny_dual_comparison)
#convertSVG("chromosome.svg", device = "png")

In [33]:
karyotype_dual_comparison <- karyotype_dual_comparison
synteny_dual_comparison <- synteny_dual_comparison[1:15,]

In [29]:
write.csv(karyotype_dual_comparison, "karyotype_dual_comparison.csv", row.names=FALSE)
write.csv(synteny_dual_comparison, "synteny_dual_comparison.csv", row.names=FALSE)